# TASK

Consider the large scale Twitter friendship dataset available at Twitter Friends (kaggle.com) of full
Twitter user profile data (40K users), including friendship relationship. We want to explore this
friendship relation to construct a network graph where User IDs are nodes and a directed edge from
IDx to IDy if IDy is listed as a friend of IDx.

1. Use NetworkX to display the corresponding network, suggest an approach to visualize this
dense network using visualization tool of your choice (NetworkX is not ideal for dense
graphs). Save the adjacency matrix of this graph in a separate file.
2. Write a script that uses NetworkX functions to calculate diameter, average clustering
coefficient and average path length of the network.
3. Write a script that plots the degree centrality distribution and closeness centrality
distribution.
4. We want to test the extent to which the centrality distributions in 3) fit a power law
distribution. You may inspire from the implementation in powerlaw · PyPI of the power-law
distribution, or can use alternative one of your choice. It is important to quantify the
goodness of fit using p-value. Typically, when p-value is greater than 10%, we can state that
power-law is a plausible fit to the (distribution) data.
5. Write a script that calculates the number of triangles in the network.
6. Write a script that identifies the largest strongly connected component, second largest, third
largest. Display each component (If there is more than one component for a given case,
then draw one at random).
7. Write a script that calculates the shortest distance between any pair of the strongly
connected components in 5). Present the result in a table. Comment on the separability
between the above components.
8. We want to identify relevant communities from the subnetwork graph corresponding to the
largest, second largest and third largest component. For this purpose, use Label propagation
algorithm implementation in NetworkX to identify the main communities. Write a script
that uses different color for each community and visualize the above graph with the detected
communities. Use the appropriate function in NetworkX to compute the separation among
the various communities and any other related quality measures. Comment on the quality of
the partition.
9. We want to evaluate the evolution of the triangles (transitivity relation in the network). For
this purpose, we consider time increment as an accumulation of one thousand successive
rows in the original dataset. Suggest a script that calculates the evolution of the proportion
of triangles (number of triangles over the total number of nodes up to time t, basically t=1,
2,…,40), and draws the corresponding graph.
10. From 9), write a script that identifies instances of triplet nodes A, B, C such at time t A is
connected to B and B is connected to C but A is not connected to C, while in time t+1, A
becomes connected to C. Check for theses instances whether the link prediction using
common neighbor (probability A being connected to C increases with the number of
common neighbours between A and B). Write the result in a table.
11. Suggest appropriate literature to comment on the various findings and explore the limitation
of the reasoning pipeline.

The data contains the following columns:
- avatar: URL to the profile picture
- followerCount: the number of followers of this user
- friendsCount: the number of people following this user
- friendName: stores the @name (without the '@') of the user (beware this name can be changed by the user)
- id: user ID, this number can not change (you can retrieve screen name with this service: https://tweeterid.com/)
- friends: the list of IDs the user follows (data stored is IDs of users followed by this user)
- lang: the language declared by the user (in this dataset there is only "en" (english))
- lastSeen: the time stamp of the date when this user have post his last tweet
- tags: the hashtags (with or without #) used by the user. It's the "trending topic" the user tweeted about
- tweetID: ID of the last tweet posted by this user

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import scipy

In [ ]:
# open the csv file
with open('data.csv', 'r') as f:
    data = f.readlines()

In [ ]:
def create_graph(data):
    G = nx.Graph()
    for line in data:
        line = line.strip().split(',')
        id = line[0].strip('"').strip()
        screenName = line[1].strip('" ')
        tags = line[2].strip('[] ').replace('"', '')
        avatar = line[3].strip('" ')
        followersCount = line[4]
        friendsCount = line[5]
        lang = line[6].strip('" ')
        lastSeen = line[7]
        tweetId = line[8].replace('"', '').strip()
        friends = line[9][1:-1].split(', ')
        # strip the quotes
        friends = [friend.replace('"', '').strip() for friend in friends]
        G.add_node(id, screenName=screenName, tags=tags, avatar=avatar, followersCount=followersCount, friendsCount=friendsCount, lang=lang, lastSeen=lastSeen, tweetId=tweetId)
        for friend in friends:
            G.add_edge(id, friend)
    print(f'Number of nodes: {G.number_of_nodes()} Number of edges: {G.number_of_edges()}')
    return G

G = create_graph(data)

# create a visualization using Gephi
# export the graph to a GEXF file
# nx.write_gexf(G, 'graph.gexf')
# print('Graph exported to graph.gexf')
# open the file in Gephi and visualize the graph

# Save the adjacency matrix
adj_matrix = nx.adjacency_matrix(G)
scipy.sparse.save_npz('adj_matrix.npz', adj_matrix)

# Visualize the graph using NetworkX (not ideal for dense graphs)
# draw only the first 1000 nodes, use screenName as labels, node size = 100
G = G.subgraph(list(G.nodes())[:100])
pos = nx.spring_layout(G)
plt.figure(figsize=(20, 20))
nx.draw(G, pos, with_labels=True, labels=nx.get_node_attributes(G, 'screenName'), node_size=80, font_size=6, linewidths=0.1, edge_color='gray')
plt.show()

